# DQN again, but with more information

Before abandoning DQN for a more full-featured approach, I want to try to apply it to a more informative environment: one that feeds the dynamic and the video frames, along with the patch data.

In [ ]:
# TODO make env (like markovian)
# TODO code things up, like dqn notebook